# Content

- Address Overfitting (Regularization, Early Stopping)
    - Conclusion from the Results
        - Overview of Metrics
        - Training Set Classification Report (old vs new)
        - Test Set Classification Report (old vs new)
        - Key Insights
        - Recommendations
- Address Underfitting (RandomizedSearchCV with Keras)
    - Search over different dropout rates and L2 regularization strengths
        - Training Set Classification Report (old vs new)
        - Test Set Classification Report (old vs new)
        - Conclusion
- Further Fine-tuning
    - Search over different dropout, L2 reg, activation, epochs, and batch_size
        - Training Set Classification Report (old vs new)
        - Test Set Classification Report (old vs new)
        - Conclusion
        - Save the model with best parameters

# Address Overfitting (Regularization, Early Stopping)

**Address Overfitting**:
   - **Regularization**: Implement regularization techniques such as dropout, L1/L2 regularization to prevent overfitting.
   - **Early Stopping**: Use early stopping during training to halt training once the model's performance on a validation set stops improving.

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Set seeds for reproducibility
import tensorflow as tf
import random
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load the dataset
df = pd.read_csv('train_split_dimentionality_reducted.csv')

# Separate features and target columns
X = df.drop(['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D'], axis=1)
y = df[['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the ANN model with regularization and dropout
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 output nodes for the 4 target classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])



C:\Users\dziya\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.3085 - loss: 2.1615 - val_accuracy: 0.4346 - val_loss: 1.6651
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4070 - loss: 1.6365 - val_accuracy: 0.4592 - val_loss: 1.4336
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4341 - loss: 1.4374 - val_accuracy: 0.4649 - val_loss: 1.3290
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4472 - loss: 1.3492 - val_accuracy: 0.4869 - val_loss: 1.2744
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4503 - loss: 1.3000 - val_accuracy: 0.4788 - val_loss: 1.2473
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4611 - loss: 1.2693 - val_accuracy: 0.4820 - val_loss: 1.2285
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4739 - loss: 1.2519 - val_accuracy: 0.4902 - val_loss: 1.2146
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4781 - loss: 1.2442 - val_accuracy: 0

In [19]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print('Train loss:', train_loss)
print('Train accuracy:', train_accuracy)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Predict the classes for training data
y_train_pred = model.predict(X_train)
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_train_labels = np.argmax(np.array(y_train), axis=1)

# Predict the classes for test data
y_test_pred = model.predict(X_test)
y_test_pred_labels = np.argmax(y_test_pred, axis=1)
y_test_labels = np.argmax(np.array(y_test), axis=1)

# Print the classification report for the training set
target_names = ['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']
print("Training Set Classification Report")
print(classification_report(y_train_labels, y_train_pred_labels, target_names=target_names))

# Print the classification report for the test set
print("Test Set Classification Report")
print(classification_report(y_test_labels, y_test_pred_labels, target_names=target_names))


153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4951 - loss: 1.1679
Train loss: 1.1601309776306152
Train accuracy: 0.507557213306427
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5283 - loss: 1.1662
Test loss: 1.1638070344924927
Test accuracy: 0.5228758454322815
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Training Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.43      0.43      0.43      1230
Segmentation_B       0.41      0.23      0.30      1160
Segmentation_C       0.54      0.54      0.54      1159
Segmentation_D       0.58      0.78      0.66      1347

      accuracy                           0.51      4896
     macro avg       0.49      0.50      0.48      4896
  weighted avg       0.49      0.51      0.49      4896

Test Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.46      0.45      0.45       309
Segmentation_

## Conclusion from the Results

### Overview of Metrics

| Metric               | Previous Result | New Result                              |
|----------------------|-----------------|-----------------------------------------|
| **Training Loss**    | 0.8918          | 1.1601                                  |
| **Training Accuracy**| 0.6168          | 0.5076                                  |
| **Test Loss**        | 1.1721          | 1.1638                                  |
| **Test Accuracy**    | 0.5008          | 0.5229                                  |

### Training Set Classification Report (old vs new)

| Segmentation      | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|-------------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A    | 0.58                 | 0.54              | 0.56                | 1230               | 0.43            | 0.43         | 0.43           | 1230          |
| Segmentation_B    | 0.55                 | 0.47              | 0.51                | 1160               | 0.41            | 0.23         | 0.30           | 1160          |
| Segmentation_C    | 0.65                 | 0.59              | 0.62                | 1159               | 0.54            | 0.54         | 0.54           | 1159          |
| Segmentation_D    | 0.66                 | 0.84              | 0.74                | 1347               | 0.58            | 0.78         | 0.66           | 1347          |
| **Overall Accuracy** |                   |                   | 0.62                | 4896               |                 |              | 0.51           | 4896          |
| **Macro Avg**     | 0.61                 | 0.61              | 0.61                | 4896               | 0.49            | 0.50         | 0.48           | 4896          |
| **Weighted Avg**  | 0.61                 | 0.62              | 0.61                | 4896               | 0.49            | 0.51         | 0.49           | 4896          |

### Test Set Classification Report (old vs new)

| Segmentation      | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|-------------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A    | 0.42                 | 0.38              | 0.40                | 309                | 0.46            | 0.45         | 0.45           | 309           |
| Segmentation_B    | 0.39                 | 0.34              | 0.36                | 283                | 0.44            | 0.25         | 0.32           | 283           |
| Segmentation_C    | 0.59                 | 0.53              | 0.56                | 292                | 0.57            | 0.57         | 0.57           | 292           |
| Segmentation_D    | 0.56                 | 0.72              | 0.63                | 340                | 0.57            | 0.78         | 0.66           | 340           |
| **Overall Accuracy** |                   |                   | 0.50                | 1224               |                 |              | 0.52           | 1224          |
| **Macro Avg**     | 0.49                 | 0.49              | 0.49                | 1224               | 0.51            | 0.51         | 0.50           | 1224          |
| **Weighted Avg**  | 0.49                 | 0.50              | 0.49                | 1224               | 0.51            | 0.52         | 0.51           | 1224          |

### Key Insights

1. **Overfitting Mitigation**: 
   - The regularization techniques and early stopping appear to have reduced overfitting, as indicated by the more consistent performance between the training and test sets. However, the overall accuracy has decreased, suggesting that the model may now be underfitting.

2. **Class Performance**:
   - **Segmentation_D**: Maintains relatively high precision, recall, and F1-score, indicating the model's robustness in predicting this class.
   - **Segmentation_B**: Shows a drop in recall and F1-score, suggesting that the model still struggles to correctly identify instances of this class.

3. **Model Performance**:
   - **Training Performance**: The new model shows lower performance on the training set compared to the previous one, indicating the impact of regularization.
   - **Test Performance**: The new model's performance on the test set is slightly lower, which might be an indicator of underfitting due to the regularization being too strong or the need for more epochs.

### Recommendations

1. **Further Tuning Regularization**:
   - Adjust the dropout rate and L2 regularization strength to find a balance between overfitting and underfitting. Experiment with dropout rates between 0.2 and 0.5 and L2 regularization values between 0.001 and 0.01.

2. **Extended Training**:
   - Increase the number of epochs with early stopping to ensure the model has sufficient time to converge.

3. **Hyperparameter Tuning**:
   - Perform hyperparameter tuning using GridSearchCV or RandomizedSearchCV to find the optimal set of hyperparameters for the model.

4. **Model Complexity**:
   - Experiment with different types of layers, to improve the model's capacity to learn from the data.

5. **Cross-Validation**:
   - Implement cross-validation to ensure that the model's performance is consistent across different subsets of the data.

By addressing these areas, we can further improve the model's performance and its ability to generalize to unseen data.
odel's performance and its ability to generalize to unseen data.


# Address Underfitting (RandomizedSearchCV with Keras)

## Search over different dropout rates and L2 regularization strengths

what changed:
- Search over different dropout rates and L2 regularization strengths added
- Early stopping patience changed from 10 to 20

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Set seeds for reproducibility
import tensorflow as tf
import random
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load the dataset
df = pd.read_csv('train_split_dimentionality_reducted.csv')

# Separate features and target columns
X = df.drop(['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D'], axis=1)
y = df[['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y to categorical
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the function to create the model
def create_model(dropout_rate=0.2, l2_reg=0.01):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model using the KerasClassifier
model = KerasClassifier(model=create_model, epochs=50, batch_size=32, verbose=0)

# Define the grid of hyperparameters to search
param_dist = {
    'model__dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'model__l2_reg': [0.001, 0.005, 0.01, 0.05]
}

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, scoring='accuracy', cv=3, 
                                   verbose=1, random_state=42, n_jobs=-1)

# Fit the RandomizedSearchCV
random_search_result = random_search.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Print the best parameters and best score
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Train the best model
best_params = random_search_result.best_params_
best_model = create_model(dropout_rate=best_params['model__dropout_rate'], l2_reg=best_params['model__l2_reg'])
history = best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best: 0.510825 using {'model__l2_reg': 0.005, 'model__dropout_rate': 0.4}


In [27]:
# Evaluate the model on the training set
train_loss, train_accuracy = best_model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', train_loss)
print('Train accuracy:', train_accuracy)

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Predict on the training set
y_train_pred = best_model.predict(X_train)

# Convert the predicted probabilities to class labels for the training set
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_train_labels = np.argmax(y_train, axis=1)

# Print the classification report for the training set
print("Training Set Classification Report")
print(classification_report(y_train_labels, y_train_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))

# Predict on the test set
y_test_pred = best_model.predict(X_test)

# Convert the predicted probabilities to class labels for the test set
y_test_pred_labels = np.argmax(y_test_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Print the classification report for the test set
print("Test Set Classification Report")
print(classification_report(y_test_labels, y_test_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))


Train loss: 1.126171588897705
Train accuracy: 0.5255310535430908
Test loss: 1.1389477252960205
Test accuracy: 0.523692786693573
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Training Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.45      0.46      0.46      1230
Segmentation_B       0.44      0.26      0.33      1160
Segmentation_C       0.57      0.53      0.55      1159
Segmentation_D       0.58      0.81      0.68      1347

      accuracy                           0.53      4896
     macro avg       0.51      0.52      0.50      4896
  weighted avg       0.51      0.53      0.51      4896

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Test Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.43      0.44      0.43       309
Segmentation_B       0.44      0.26      0.33       283
Segmentation_C       0.60      0.55      0.57       292
Segmentation_D       0.58      0.80      0.67     

## Training Set Classification Report (old vs new)

| Segmentation  | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|---------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A| 0.43                 | 0.43              | 0.43                | 1230               | 0.45            | 0.46         | 0.46           | 1230          |
| Segmentation_B| 0.41                 | 0.23              | 0.30                | 1160               | 0.44            | 0.26         | 0.33           | 1160          |
| Segmentation_C| 0.54                 | 0.54              | 0.54                | 1159               | 0.57            | 0.53         | 0.55           | 1159          |
| Segmentation_D| 0.58                 | 0.78              | 0.66                | 1347               | 0.58            | 0.81         | 0.68           | 1347          |
| Accuracy      |                      |                   | 0.51                | 4896               |                 |              | 0.53           | 4896          |
| Macro Avg     | 0.49                 | 0.50              | 0.48                | 4896               | 0.51            | 0.52         | 0.50           | 4896          |
| Weighted Avg  | 0.49                 | 0.51              | 0.49                | 4896               | 0.51            | 0.53         | 0.51           | 4896          |

## Test Set Classification Report (old vs new)

| Segmentation  | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|---------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A| 0.46                 | 0.45              | 0.45                | 309                | 0.43            | 0.44         | 0.43           | 309           |
| Segmentation_B| 0.44                 | 0.25              | 0.32                | 283                | 0.44            | 0.26         | 0.33           | 283           |
| Segmentation_C| 0.57                 | 0.57              | 0.57                | 292                | 0.60            | 0.55         | 0.57           | 292           |
| Segmentation_D| 0.57                 | 0.78              | 0.66                | 340                | 0.58            | 0.80         | 0.67           | 340           |
| Accuracy      |                      |                   | 0.52                | 1224               |                 |              | 0.52           | 1224          |
| Macro Avg     | 0.51                 | 0.51              | 0.50                | 1224               | 0.51            | 0.51         | 0.50           | 1224          |
| Weighted Avg  | 0.51                 | 0.52              | 0.51                | 1224               | 0.51            | 0.52         | 0.51           | 1224          |

### Overview of Metrics

| Metric               | Previous Result | New Result                              |
|----------------------|-----------------|-----------------------------------------|
| **Training Loss**    | 1.1601          | 1.1262                                  |
| **Training Accuracy**| 0.5076          | 0.5255                                  |
| **Test Loss**        | 1.1638          | 1.1389                                  |
| **Test Accuracy**    | 0.5229          | 0.5237                                  |

## Conclusion

- **Training Set**:
  - The new model shows a slight improvement in overall accuracy from 0.51 to 0.53.
  - Precision, recall, and F1-scores have slightly increased for Segmentation_A and Segmentation_B but decreased slightly for Segmentation_C and Segmentation_D.
  - The macro and weighted averages for precision, recall, and F1-score have also improved slightly.

- **Test Set**:
  - The overall accuracy on the test set increased slightly from 0.52 to 0.53.
  - Precision, recall, and F1-scores have increased for Segmentation_C and Segmentation_D but slightly decreased for Segmentation_A and Segmentation_B.
  - Macro and weighted averages for precision, recall, and F1-score show slight improvements.

The slight improvements in both training and test set metrics indicate that the chosen best hyperparameters (dropout rate and L2 regularization) have positively impacted its performance, making it more generalizable and defying the segments correctly.
t more generalizable and better at classifying the segments correctly.


## Further Fine-tuning

### Search over different dropout, L2 reg, activation, epochs, and batch_size

Updated code that includes activation, epochs, and batch_size in the random search, and uses all available cores.
Also, patience in reduce_lr was increased from 5 to 10

In [33]:
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.metrics import classification_report

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Custom callback for logging
class LoggingCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logging.info(f"Epoch {epoch + 1} completed. Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}")

# Load the dataset
df = pd.read_csv('train_split_dimentionality_reducted.csv')

# Separate features and target columns
X = df.drop(['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D'], axis=1)
y = df[['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y to categorical
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the function to create the model
def create_model(dropout_rate=0.2, l2_reg=0.01, learning_rate=0.001, activation='relu'):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(64, activation=activation, kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation=activation, kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model using the KerasClassifier
model = KerasClassifier(model=create_model, verbose=0)

# Define the grid of hyperparameters to search
param_dist = {
    'model__dropout_rate': [0.2, 0.4],
    'model__l2_reg': [0.001, 0.01],
    'model__learning_rate': [0.001, 0.01, 0.1],
    'model__activation': ['relu', 'tanh'],
    'epochs': [10, 50, 100],
    'batch_size': [32, 64]
}

# Define early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)
logging_callback = LoggingCallback()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, scoring='accuracy', cv=3, 
                                   verbose=1, random_state=42, n_jobs=-1)

# Fit the RandomizedSearchCV
random_search_result = random_search.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr, logging_callback])

# Print the best parameters and best score
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Train the best model
best_params = random_search_result.best_params_
best_model = create_model(dropout_rate=best_params['model__dropout_rate'], l2_reg=best_params['model__l2_reg'], learning_rate=best_params['model__learning_rate'], activation=best_params['model__activation'])
history = best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr, logging_callback], verbose=0)



Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best: 0.507966 using {'model__learning_rate': 0.001, 'model__l2_reg': 0.01, 'model__dropout_rate': 0.2, 'model__activation': 'relu', 'epochs': 100, 'batch_size': 32}


2024-07-18 14:37:39,758 - INFO - Epoch 1 completed. Loss: 1.8887606859207153, Accuracy: 0.40992647409439087, Val Loss: 1.583421230316162, Val Accuracy: 0.4591503143310547
2024-07-18 14:37:41,678 - INFO - Epoch 2 completed. Loss: 1.4756624698638916, Accuracy: 0.45812907814979553, Val Loss: 1.3693695068359375, Val Accuracy: 0.4820261299610138
2024-07-18 14:37:42,963 - INFO - Epoch 3 completed. Loss: 1.337983250617981, Accuracy: 0.464665025472641, Val Loss: 1.2838596105575562, Val Accuracy: 0.47875815629959106
2024-07-18 14:37:44,003 - INFO - Epoch 4 completed. Loss: 1.2702945470809937, Accuracy: 0.47732841968536377, Val Loss: 1.2408149242401123, Val Accuracy: 0.49754902720451355
2024-07-18 14:37:45,370 - INFO - Epoch 5 completed. Loss: 1.246335506439209, Accuracy: 0.4818218946456909, Val Loss: 1.21883225440979, Val Accuracy: 0.49754902720451355
2024-07-18 14:37:46,678 - INFO - Epoch 6 completed. Loss: 1.225836157798767, Accuracy: 0.48631536960601807, Val Loss: 1.2066571712493896, Val Acc

In [34]:
# Evaluate the model on the training set
train_loss, train_accuracy = best_model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', train_loss)
print('Train accuracy:', train_accuracy)

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Predict on the training set
y_train_pred = best_model.predict(X_train)

# Convert the predicted probabilities to class labels for the training set
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_train_labels = np.argmax(y_train, axis=1)

# Print the classification report for the training set
print("Training Set Classification Report")
print(classification_report(y_train_labels, y_train_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))

# Predict on the test set
y_test_pred = best_model.predict(X_test)

# Convert the predicted probabilities to class labels for the test set
y_test_pred_labels = np.argmax(y_test_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Print the classification report for the test set
print("Test Set Classification Report")
print(classification_report(y_test_labels, y_test_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))


Train loss: 1.104968547821045
Train accuracy: 0.5371732115745544
Test loss: 1.122696042060852
Test accuracy: 0.5220588445663452
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Training Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.47      0.49      0.48      1230
Segmentation_B       0.44      0.32      0.37      1160
Segmentation_C       0.58      0.54      0.56      1159
Segmentation_D       0.61      0.76      0.68      1347

      accuracy                           0.54      4896
     macro avg       0.52      0.53      0.52      4896
  weighted avg       0.53      0.54      0.53      4896

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Test Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.43      0.47      0.45       309
Segmentation_B       0.42      0.27      0.33       283
Segmentation_C       0.58      0.56      0.57       292
Segmentation_D       0.60      0.75      0.67     

## Training Set Classification Report (old vs new)

| Segmentation  | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|---------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A| 0.45                 | 0.46              | 0.46                | 1230               | 0.47            | 0.49         | 0.48           | 1230          |
| Segmentation_B| 0.44                 | 0.26              | 0.33                | 1160               | 0.44            | 0.32         | 0.37           | 1160          |
| Segmentation_C| 0.57                 | 0.53              | 0.55                | 1159               | 0.58            | 0.54         | 0.56           | 1159          |
| Segmentation_D| 0.58                 | 0.81              | 0.68                | 1347               | 0.61            | 0.76         | 0.68           | 1347          |
| **Accuracy**  |                      |                   | 0.53                | 4896               |                 |              | 0.54           | 4896          |
| **Macro Avg** | 0.51                 | 0.52              | 0.50                | 4896               | 0.52            | 0.53         | 0.52           | 4896          |
| **Weighted Avg** | 0.51              | 0.53              | 0.51                | 4896               | 0.53            | 0.54         | 0.53           | 4896          |

## Test Set Classification Report (old vs new)

| Segmentation  | Precision (Previous) | Recall (Previous) | F1-Score (Previous) | Support (Previous) | Precision (New) | Recall (New) | F1-Score (New) | Support (New) |
|---------------|----------------------|-------------------|---------------------|--------------------|-----------------|--------------|----------------|---------------|
| Segmentation_A| 0.43                 | 0.44              | 0.43                | 309                | 0.43            | 0.47         | 0.45           | 309           |
| Segmentation_B| 0.44                 | 0.26              | 0.33                | 283                | 0.42            | 0.27         | 0.33           | 283           |
| Segmentation_C| 0.60                 | 0.55              | 0.57                | 292                | 0.58            | 0.56         | 0.57           | 292           |
| Segmentation_D| 0.58                 | 0.80              | 0.67                | 340                | 0.60            | 0.75         | 0.67           | 340           |
| **Accuracy**  |                      |                   | 0.52                | 1224               |                 |              | 0.52           | 1224          |
| **Macro Avg** | 0.51                 | 0.51              | 0.50                | 1224               | 0.51            | 0.51         | 0.50           | 1224          |
| **Weighted Avg** | 0.51              | 0.52              | 0.51                | 1224               | 0.51            | 0.52         | 0.51           | 1224          |

## Conclusion

- **Training Set**:
  - The overall accuracy on the training set has slightly increased from 0.53 to 0.54.
  - Precision, recall, and F1-scores have slightly improved for most segments, with Segmentation_A and Segmentation_D showing the most notable improvements.
  - The macro and weighted averages for precision, recall, and F1-score have shown slight improvements.

- **Test Set**:
  - The overall accuracy on the test set has remained stable at 0.52.
  - Precision, recall, and F1-scores have remained relatively stable for Segmentation_B, Segmentation_C, and Segmentation_D, with slight improvements in Segmentation_A.
  - Macro and weighted averages for precision, recall, and F1-score have remained stable.

The new model, with adjusted hyperparameters and learning rate adjustments, has shown a slight improvement in training performance but has maintained with slight improvements in Segmentation_A s

**Best parameters are:**
- {'model__learning_rate': 0.001, 'model__l2_reg': 0.01, 'model__dropout_rate': 0.2, 'model__activation': 'relu', 'epochs': 100, 'batch_size': 32}
- early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
- reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)ficant improvements in performance.


## Save the model with best parameters

Here is modified code with the best parameters applied directly and the model saved for future use:
{'model__learning_rate': 0.001, 'model__l2_reg': 0.01, 'model__dropout_rate': 0.2, 'model__activation': 'relu', 'epochs': 100, 'batch_size': 32}

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import random

# Set seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load the dataset
df = pd.read_csv('train_split_dimentionality_reducted.csv')

# Separate features and target columns
X = df.drop(['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D'], axis=1)
y = df[['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y to categorical
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the function to create the model with the best parameters
def create_model(dropout_rate=0.2, l2_reg=0.01, learning_rate=0.001, activation='relu'):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(64, activation=activation, kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation=activation, kernel_regularizer=regularizers.l2(l2_reg)),
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and compile the model with the best parameters
model = create_model(dropout_rate=0.2, l2_reg=0.01, learning_rate=0.001, activation='relu')

# Define early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=0)

# Save the model
model.save('pretrained_ann_classification_model.keras')
print('Model saved')

Model saved


In [45]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', train_loss)
print('Train accuracy:', train_accuracy)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Predict on the training set
y_train_pred = model.predict(X_train)

# Convert the predicted probabilities to class labels for the training set
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_train_labels = np.argmax(y_train, axis=1)

# Print the classification report for the training set
print("Training Set Classification Report")
print(classification_report(y_train_labels, y_train_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))

# Predict on the test set
y_test_pred = model.predict(X_test)

# Convert the predicted probabilities to class labels for the test set
y_test_pred_labels = np.argmax(y_test_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Print the classification report for the test set
print("Test Set Classification Report")
print(classification_report(y_test_labels, y_test_pred_labels, target_names=['Segmentation_A', 'Segmentation_B', 'Segmentation_C', 'Segmentation_D']))


Train loss: 1.1057605743408203
Train accuracy: 0.5334967374801636
Test loss: 1.1231225728988647
Test accuracy: 0.5261437892913818
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Training Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.46      0.47      0.47      1230
Segmentation_B       0.43      0.32      0.36      1160
Segmentation_C       0.57      0.56      0.56      1159
Segmentation_D       0.62      0.76      0.68      1347

      accuracy                           0.53      4896
     macro avg       0.52      0.53      0.52      4896
  weighted avg       0.52      0.53      0.52      4896

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Test Set Classification Report
                precision    recall  f1-score   support

Segmentation_A       0.44      0.46      0.45       309
Segmentation_B       0.42      0.29      0.34       283
Segmentation_C       0.58      0.57      0.57       292
Segmentation_D       0.61      0.75      0.67   

**Why result is different when I use best parameters directly from the run where best parameters were found?**
When we directly use the best parameters from a hyperparameter tuning run, you might expect the results to be similar or even better. However, there can be several reasons for discrepancies in the results:

- Random Initialization: Neural networks, especially with Keras and TensorFlow, involve random initialization of weights. Even with the same hyperparameters, different initial weights can lead to different convergence behavior and thus different final results.

- Stochastic Nature of Training: The training process involves random shuffling of data and can behave differently in different runs due to the stochastic nature of optimization algorithms like Adam. This can lead to different final results even with the same hyperparameters.

- Regularization: Dropout is a stochastic regularization technique. The neurons are randomly dropped during training, leading to variations in the training process and final model performance.

- Early Stopping: If early stopping is used, the exact epoch at which training stops can vary slightly between runs, leading to different model weights and performance.

    To reduce these discrepancies, you can set random seeds for reproducibility. This can help ensure that the weight initialization and data shuffling are consistent across different runs. 